<a href="https://colab.research.google.com/github/AhmedAlziadi/1000MLProjects/blob/main/MingFang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
import argparse
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from sklearn.metrics import adjusted_rand_score as ari_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.nn import Linear
from Attention import CAM
from utils_Adam_multiview import MnistDataset, cluster_acc

import os    #获取当前工作路径
import pandas as pd   #将数据保存至相应文件中

from pytorchtools import EarlyStopping
import random
import warnings
warnings.filterwarnings("ignore")
def setup_seed(seed):  # 设置随机种子，用于实验结果复现
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
setup_seed(2023)

ModuleNotFoundError: ignored

In [ ]:
class AE(nn.Module):

    def __init__(self, n_enc_1, n_enc_2, n_enc_3, n_dec_1, n_dec_2, n_dec_3,
                 n_input, n_z):
        super(AE, self).__init__()

        # encoder
        self.enc_1 = Linear(n_input, n_enc_1)
        self.enc_2 = Linear(n_enc_1, n_enc_2)
        self.enc_3 = Linear(n_enc_2, n_enc_3)

        self.z_layer = Linear(n_enc_3, n_z)

        # decoder
        self.dec_1 = Linear(n_z, n_dec_1)
        self.dec_2 = Linear(n_dec_1, n_dec_2)
        self.dec_3 = Linear(n_dec_2, n_dec_3)

        self.x_bar_layer = Linear(n_dec_3, n_input)

    def forward(self, x):

        # encoder
        enc_h1 = F.relu(self.enc_1(x))
        enc_h2 = F.relu(self.enc_2(enc_h1))
        enc_h3 = F.relu(self.enc_3(enc_h2))

        z = self.z_layer(enc_h3)

        # decoder
        dec_h1 = F.relu(self.dec_1(z))
        dec_h2 = F.relu(self.dec_2(dec_h1))
        dec_h3 = F.relu(self.dec_3(dec_h2))
        x_bar = self.x_bar_layer(dec_h3)

        return x_bar, z


In [ ]:
!pip install torchvision


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\site-packages\pip\_vendor\urllib3\response.py", line 425, in _error_catcher
    yield
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\site-packages\pip\_vendor\urllib3\response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\http\client.py", line 457, in read
    n = self.readinto(b)
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\http\client.py", line 501, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\socket.py", line 589, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Ahmed\anaconda3\anaconda.3.1\lib\ssl.py", line 1071, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\Ah

In [ ]:
class MAE(nn.Module):

    def __init__(self, n_enc_1 , n_enc_2, n_enc_3, n_dec_1, n_dec_2, n_dec_3, n_input, in_channels, n_z):
        super(MAE, self).__init__()
        # self.n_enc_1 = n_enc_1,
        # self.n_enc_2 = n_enc_2,
        # self.n_enc_3 = n_enc_3,
        # self.n_dec_1 = n_dec_1,
        # self.n_dec_2 = n_dec_2,
        # self.n_dec_3 = n_dec_3,
        # self.n_input = n_input,
        # self.n_z = n_z,
        self.ae1 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae2 = AE(
             n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae3 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae4 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae5 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae6 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae7 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)
        self.ae8 = AE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            n_z=n_z)

        self.cam = CAM(in_channels=in_channels)

    def forward(self, x):
        x_bar1, z1 = self.ae1(x[:,0,:])
        x_bar2, z2 = self.ae2(x[:,1,:])
        x_bar3, z3 = self.ae3(x[:,2,:])
        x_bar4, z4 = self.ae4(x[:,3,:])
        x_bar5, z5 = self.ae5(x[:,4,:])
        x_bar6, z6 = self.ae6(x[:,5,:])
        x_bar7, z7 = self.ae7(x[:,6,:])
        x_bar8, z8 = self.ae8(x[:,7,:])
#         print(f'z1.shape: {z1.shape}')
        loss = F.mse_loss(x_bar1, x[:, 0, :]) + F.mse_loss(x_bar2, x[:, 1, :]) + F.mse_loss(x_bar3, x[:, 2, :])
        + F.mse_loss(x_bar4, x[:, 3, :]) + F.mse_loss(x_bar5, x[:, 4, :]) + F.mse_loss(x_bar6, x[:, 5, :])
        + F.mse_loss(x_bar7, x[:, 6, :]) + F.mse_loss(x_bar8, x[:, 7, :])
        # 多视图融合
        z_cat = torch.cat((z1.unsqueeze(1), z2.unsqueeze(1), z3.unsqueeze(1), z4.unsqueeze(1),
                           z5.unsqueeze(1), z6.unsqueeze(1), z7.unsqueeze(1), z8.unsqueeze(1)), dim=1)
#         print(f'z_cat.shape: {z_cat.shape}')
#         z = torch.sum(z_cat,dim=1)
        Cam = self.cam(z_cat)
        z = Cam[0].squeeze(1)
#         print(f'z.shape: {z.shape}')
        c_weights = Cam[1]
#         print(f'c_weights.shape: {c_weights.shape}')
#         print(net.state_dict()['module.conv1.1.bias'])  # 填该层参数名

        embedding = [z1, z2, z3, z4, z5, z6, z7, z8]
        # Loss_constraint1,embedding之间的MSE，不过这里符号要取负,考虑视图特征之间的多样性
        L_con1 = 0
        for i in range(len(embedding)):
            for j in range(i+1, len(embedding)):
                L_con1 += F.mse_loss(embedding[i], embedding[j])
        # Loss_constraint2,embedding与一致性embedding之间的MSE,考虑视图特征之间的一致性
        L_con2 = 0
        for i in range(len(embedding)):
            L_con2 += F.mse_loss(z, embedding[i])

        return z, loss, L_con1, L_con2, c_weights


In [ ]:
class IDEC(nn.Module):

    def __init__(self,
                 n_enc_1,
                 n_enc_2,
                 n_enc_3,
                 n_dec_1,
                 n_dec_2,
                 n_dec_3,
                 n_input,
                 n_z,
                 in_channels,
                 n_clusters,
                 alpha=1,
                 pretrain_path='ae_Adam_multiview.pkl'):
        super(IDEC, self).__init__()
        self.alpha = 1.0
        self.pretrain_path = pretrain_path

#         self.ae = AE(
#             n_enc_1=n_enc_1,
#             n_enc_2=n_enc_2,
#             n_enc_3=n_enc_3,
#             n_dec_1=n_dec_1,
#             n_dec_2=n_dec_2,
#             n_dec_3=n_dec_3,
#             n_input=n_input,
#             n_z=n_z)

        self.ae = MAE(
            n_enc_1=n_enc_1,
            n_enc_2=n_enc_2,
            n_enc_3=n_enc_3,
            n_dec_1=n_dec_1,
            n_dec_2=n_dec_2,
            n_dec_3=n_dec_3,
            n_input=n_input,
            in_channels=in_channels,
            n_z=n_z)

#         self.cam = CAM(in_channels=in_channels)

        # cluster layer
        self.cluster_layer = Parameter(torch.Tensor(n_clusters, n_z))

        torch.nn.init.xavier_normal_(self.cluster_layer.data)

    def pretrain(self, path=''):
        if path == '':
            pretrain_ae(self.ae)

        # load pretrain weights
        self.ae.load_state_dict(torch.load(self.pretrain_path))
        print('load pretrained ae from', path)


#     def forward(self, x):

#         x_bar, z = self.ae(x)
#         # cluster
#         q = 1.0 / (1.0 + torch.sum(
#             torch.pow(z.unsqueeze(1) - self.cluster_layer, 2), 2) / self.alpha)
#         q = q.pow((self.alpha + 1.0) / 2.0)
#         q = (q.t() / torch.sum(q, 1)).t()
#         return x_bar, q

#     def forward(self, x):
#         x_bar1, z1 = self.ae(x[:, 0, :])
#         x_bar2, z2 = self.ae(x[:, 1, :])
#         x_bar3, z3 = self.ae(x[:, 2, :])
#         x_bar4, z4 = self.ae(x[:, 3, :])
#         x_bar5, z5 = self.ae(x[:, 4, :])
#         x_bar6, z6 = self.ae(x[:, 5, :])
#         x_bar7, z7 = self.ae(x[:, 6, :])
#         x_bar8, z8 = self.ae(x[:, 7, :])
#         x_bar = torch.concat((x_bar1,x_bar2,x_bar3,x_bar4,x_bar5,x_bar6,x_bar7,x_bar8),dim=1)

#         z = torch.concat((z1,z2,z3,z4,z5,z6,z7,z8),dim=1)

#         # cluster
#         q = 1.0 / (1.0 + torch.sum(
#             torch.pow(z.unsqueeze(1) - self.cluster_layer, 2), 2) / self.alpha)
#         q = q.pow((self.alpha + 1.0) / 2.0)
#         q = (q.t() / torch.sum(q, 1)).t()

#         return x_bar, q

    def forward(self, x):
        ae = self.ae(x)
        loss = ae[1]
        L_con1 = ae[2]
        L_con2 = ae[3]
        z = ae[0]
        c_weights = ae[4]
#         z_cat = torch.cat((ae[0].unsqueeze(1), ae[1].unsqueeze(1), ae[2].unsqueeze(1), ae[3].unsqueeze(1),
#                            ae[4].unsqueeze(1), ae[5].unsqueeze(1), ae[6].unsqueeze(1), ae[7].unsqueeze(1)), dim=1)
        # print(f"ae[0].shape:{ae[0].shape}")
        # print(f"z_cat.shape:{z_cat.shape}")
#         z = self.cam(z_cat).squeeze(1)
        # print(f"z.shape:{z.shape}")
        # cluster
        q = 1.0 / (1.0 + torch.sum(
            torch.pow(z.unsqueeze(1) - self.cluster_layer, 2), 2) / self.alpha)
        q = q.pow((self.alpha + 1.0) / 2.0)
        q = (q.t() / torch.sum(q, 1)).t()
        return z, q, loss, L_con1, L_con2, c_weights


In [ ]:
def target_distribution(q):
    weight = q**2 / q.sum(0)
    return (weight.t() / weight.sum(1)).t()

In [ ]:
def pretrain_ae(model):
    '''
    pretrain autoencoder
    '''
    train_loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)
    print(model)
    optimizer = Adam(model.parameters(), lr=args.lr)
    early_stopping = EarlyStopping(patience=10, verbose=True)
#     ACC = []
#     ARI = []
#     NMI = []
#     lambda1 = args.lambda1
#     lambda2 = args.lambda2
    for epoch in range(100):
        total_loss = 0.
#         z_all = []
        B_Y = []
        for batch_idx, (x, b_y, _) in enumerate(train_loader): # x: torch.Size([256, 8, 2048]), b_y: torch.Size([256])
            x = x.to(device)
            b_y = b_y.to(device)
            # optimizer.zero_grad()
            # x_bar, z = model(x)
            # loss = F.mse_loss(x_bar, x)
            # total_loss += loss.item()
            #
            # loss.backward()
            # optimizer.step()

            B_Y.append(b_y)
            optimizer.zero_grad()
            z, loss, L_con1, L_con2, c_weights = model(x)
            total_loss += loss.item()
#             z_all.append(z)

            loss.backward()
            optimizer.step()
#     print(z_all[-1])


#         kmeans = KMeans(n_clusters=args.n_clusters, n_init=30)
#         y_pred = kmeans.fit_predict(torch.cat(z_all).data.cpu().numpy())
#         # print(f"torch.cat(z_all).shape:{torch.cat(z_all).shape}")
#         # print(f"torch.cat(B_Y).shape:{torch.cat(B_Y).shape}")
#         # print(f"Y.shape:{Y.shape}")
#         # print(f"y_pred.shape:{y_pred.shape}")
#         acc = cluster_acc(torch.cat(B_Y).cpu().numpy(), y_pred)
#         nmi = nmi_score(torch.cat(B_Y).cpu().numpy(), y_pred)
#         ari = ari_score(torch.cat(B_Y).cpu().numpy(), y_pred)
#         print('Iter {}'.format(epoch), ':Acc {:.4f}'.format(acc),
#               ', nmi {:.4f}'.format(nmi), ', ari {:.4f}'.format(ari))

#         ACC.append(acc)
#         NMI.append(nmi)
#         ARI.append(ari)

        print("epoch {} loss={:.4f}".format(epoch,
                                            total_loss / (batch_idx + 1)))

        early_stopping(total_loss / (batch_idx + 1), model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
        torch.save(model.state_dict(), args.pretrain_path)
    print("model saved to {}.".format(args.pretrain_path))
#     return z_all[-1]

#     print(f"ACC_MAX:{max(ACC)}")
#     print(f"NMI_MAX:{max(NMI)}")
#     print(f"ARI_MAX:{max(ARI)}")

#     file = os.getcwd() + 'ae_Adam_multiview_F3.csv'    #保存文件位置，即当前工作路径下的csv文件
#     data = pd.DataFrame({'ACC_MAX': [max(ACC)], 'NMI_MAX': [max(NMI)],'ARI_MAX': [max(ARI)],
#                          'LR': [args.lr],
'Hidden_z': [args.n_z], 'Layer1': [args.layer1], 'Layer2': [args.layer2], 'gamma': [args.gamma], 'lambda1': [args.lambda1], 'lambda2': [args.lambda2]})  # 要保存的数据
#     data.to_csv(file, index=True, mode='a+')  # 数据写入，index=False表示不加索引

In [ ]:
def train_idec():

    model = IDEC(
        n_enc_1=args.layer1,
        n_enc_2=args.layer1,
        n_enc_3=args.layer2,
        n_dec_1=args.layer2,
        n_dec_2=args.layer1,
        n_dec_3=args.layer1,
        n_input=args.n_input,
        n_z=args.n_z,
        in_channels=args.in_channels,
        n_clusters=args.n_clusters,
        alpha=1.0,
        pretrain_path=args.pretrain_path).to(device)

    #  model.pretrain('ae_Adam_multiview.pkl')
    model.pretrain()

    train_loader = DataLoader(
        dataset, batch_size=args.batch_size, shuffle=False)
    optimizer = Adam(model.parameters(), lr=args.lr)

#     z_all = []

    # cluster parameter initiate
    data = dataset.x
    y = dataset.y
    data = torch.Tensor(data).to(device)
    hidden, _, _, _, _, c_weights = model(data)
#     z_all.append(hidden)
    kmeans = KMeans(n_clusters=args.n_clusters, n_init=20)
    y_pred = kmeans.fit_predict(hidden.data.cpu().numpy())
#     nmi_k = nmi_score(y_pred, y)
#     print("nmi score={:.4f}".format(nmi_k))

    acc_k = cluster_acc(y, y_pred)
    nmi_k = nmi_score(y, y_pred)
    ari_k = ari_score(y, y_pred)

    print( ':acc={:.4f}'.format(acc_k),', nmi={:.4f}'.format(nmi_k), ', ari={:.4f}'.format(ari_k))
#     print(f"z_init:{z_all[-1]}",f"z_init.shape:{z_all[-1].shape}")
#     z_init = z_all[-1].cpu().data.numpy()
#     np.savetxt('_seed2023_init_multiview_可视化F44.csv', z_init, delimiter=',')
#     file = os.getcwd() + '_seed2023_init_multiview_可视化F44.csv'    #保存文件位置，即当前工作路径下的csv文件
#     DATA = pd.DataFrame({'ACC_init': [acc_k], 'NMI_init': [nmi_k],'ARI_init': [ari_k],
# #                          'z_init': [z_all[-1]],
#                          'LR': [args.lr], 'Hidden_z': [args.n_z], 'Layer1': [args.layer1], 'Layer2': [args.layer2], 'gamma': [args.gamma], 'lambda1': [args.lambda1], 'lambda2': [args.lambda2]})  # 要保存的数据
#     DATA.to_csv(file, index=True, mode='a+')  # 数据写入，index=False表示不加索引

    hidden = None
#     x_bar = None

    y_pred_last = y_pred
    model.cluster_layer.data = torch.tensor(kmeans.cluster_centers_).to(device)

    model.train()
    ACC = []
    ARI = []
    NMI = []
#     Z_ALL = []
    C_WEIGHTS = []
    lambda1 = args.lambda1
    lambda2 = args.lambda2
    for epoch in range(10): # 200

        if epoch % args.update_interval == 0:

            Z, tmp_q, _, _, _, c_weights = model(data)

            # update target distribution p
            tmp_q = tmp_q.data
            p = target_distribution(tmp_q)

            # evaluate clustering performance
            y_pred = tmp_q.cpu().numpy().argmax(1)
            delta_label = np.sum(y_pred != y_pred_last).astype(
                np.float32) / y_pred.shape[0]
            y_pred_last = y_pred

            acc = cluster_acc(y, y_pred)
            nmi = nmi_score(y, y_pred)
            ari = ari_score(y, y_pred)
            print('Iter {}'.format(epoch), ':Acc {:.4f}'.format(acc),
                  ', nmi {:.4f}'.format(nmi), ', ari {:.4f}'.format(ari))
            ACC.append(acc)
            NMI.append(nmi)
            ARI.append(ari)
#             Z_ALL.append(Z)
            C_WEIGHTS.append(c_weights)
            # if epoch > 0 and delta_label < args.tol:
            #     print('delta_label {:.4f}'.format(delta_label), '< tol',
            #           args.tol)
            #     print('Reached tolerance threshold. Stopping training.')
            #     break
        for batch_idx, (x, _, idx) in enumerate(train_loader):

            x = x.to(device)
            idx = idx.to(device)

#             x_bar, q = model(x)

#             reconstr_loss = F.mse_loss(x_bar, x)

            _, q, re_loss, L_con1, L_con2, c_weights = model(x)

#             C_WEIGHTS.append(c_weights)
#             print(c_weights.shape)
#             print(np.array(C_WEIGHTS).shape)

            ae_loss = re_loss-lambda1*L_con1+lambda2*L_con2

            kl_loss = F.kl_div(q.log(), p[idx.long()])
            loss = args.gamma * kl_loss + ae_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()



    print(f"ACC_MAX:{max(ACC)}")
    index_max = ACC.index(max(ACC))
    print(f"NMI_MAX:{NMI[index_max]}")
    print(f"ARI_MAX:{ARI[index_max]}")
#     print(f"z_finetuning:{Z_ALL[index_max]}",f"z_finetuning.shape:{Z_ALL[index_max].shape}")
#     z_finetuning = Z_ALL[index_max].cpu().data.numpy()
#     np.savetxt('_seed2023_finetuning_multiview_可视化F44.csv', z_finetuning, delimiter=',')

#     print(f"C_weights:{C_WEIGHTS[(index_max):((index_max+1))]}")
#     print(f"C_weights:{C_WEIGHTS[(5110*index_max):(5110*(index_max+1))]}")
#     C_weights = np.array(C_WEIGHTS[(5110*(index_max+1)-10):(5110*(index_max+1))])#.reshape(5110,8)
    C_weights = np.array(C_WEIGHTS[((index_max)):(index_max+1)])#.reshape(5110,8)
    print(C_weights[0].shape)
    file = os.getcwd() + 'fffffggggg_seed2023_IDEC_multiview_通道权重F1.csv'    #保存文件位置，即当前工作路径下的csv文件
    for i in range(5110):
        data = pd.DataFrame({'ACC_MAX': [max(ACC)], 'NMI_MAX': [NMI[index_max]],'ARI_MAX': [ARI[index_max]],
#                          'C_weights': [C_weights],
                             'C1_weights': [C_weights[0][i,0,:]],'C2_weights': [C_weights[0][i,1,:]],'C3_weights': [C_weights[0][i,2,:]],'C4_weights': [C_weights[0][i,3,:]],
                             'C5_weights': [C_weights[0][i,4,:]],'C6_weights': [C_weights[0][i,5,:]],'C7_weights': [C_weights[0][i,6,:]],'C8_weights': [C_weights[0][i,7,:]],
                         'LR': [args.lr], 'Hidden_z': [args.n_z], 'Layer1': [args.layer1], 'Layer2': [args.layer2], 'gamma': [args.gamma], 'lambda1': [args.lambda1], 'lambda2': [args.lambda2]})
#     data = pd.DataFrame({'ACC_MAX': [max(ACC)], 'NMI_MAX': [NMI[index_max]],'ARI_MAX': [ARI[index_max]],'c_weights': [C_WEIGHTS[(5110*index_max):(5110*(index_max+1))]],
#                          'LR': [args.lr], 'Hidden_z': [args.n_z], 'Layer1': [args.layer1], 'Layer2': [args.layer2], 'gamma': [args.gamma], 'lambda1': [args.lambda1], 'lambda2': [args.lambda2]})  # 要保存的数据
        data.to_csv(file, index=True, mode='a+')  # 数据写入，index=False表示不加索引
        # 打印所有参数
#     for name, parameters in model.named_parameters():
#         print(name, ':', parameters)

In [ ]:
if __name__ == "__main__":
    # 超参数设置    # F1:100 0.001 0.001    F2:100 0.001 0.001    F3:1000 0.01 0.1    F4:1000 0.001 0.1
    setup_seed(2023)
    for gamma in [100]: # 1000,100,10,1,0.1,0.01,0.001
        for lr in [0.0001]:
            for h_z in [20]: # 10, 20, 40, 80, 160
                for layer1 in [500]: # 500, 800, 1200, 1600, 2000, 2400
                    for layer2 in [1600]: # 1000, 1600, 2400, 3200, 4000, 4800
                        for lambda1 in [0.001]: # 0.001,0.01,0.1,1,10,100
                            for lambda2 in [0.001]: # 0.001,0.01,0.1,1,10,100
                                for i in range(10):

                                    parser = argparse.ArgumentParser(
                                        description='train',
                                        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
                                    parser.add_argument('--lambda1', type=float, default=lambda1)
                                    parser.add_argument('--lambda2', type=float, default=lambda2)
                                    parser.add_argument('--layer1', type=int, default=layer1)
                                    parser.add_argument('--layer2', type=int, default=layer2)
                                    parser.add_argument('--lr', type=float, default=lr)
                                    parser.add_argument('--in_channels', type=int, default=8)
                                    parser.add_argument('--n_clusters', default=5, type=int)
                                    parser.add_argument('--batch_size', default=256, type=int)
                                    parser.add_argument('--n_z', default=h_z, type=int)
                                    parser.add_argument('--dataset', type=str, default='mnist')
                                    parser.add_argument('--pretrain_path', type=str, default='ae_Adam_multiview')
                                    parser.add_argument(
                                        '--gamma',
                                        default=gamma,
                                        type=float,
                                        help='coefficient of clustering loss')
                                    parser.add_argument('--update_interval', default=1, type=int)
                                    parser.add_argument('--tol', default=0.001, type=float)
                                    args = parser.parse_known_args()[0]
                                    args.cuda = torch.cuda.is_available()
                                    print("use cuda: {}".format(args.cuda))
    #                                     device = torch.device("cuda" if args.cuda else "cpu")
                                    device = torch.device('cuda:1')
                                    if args.dataset == 'mnist':
                                        args.pretrain_path = 'ae_Adam_multiview.pkl'
                                        args.n_clusters = 5
                                        args.n_input = 2048
                                        dataset = MnistDataset()
                                    print(args)
                                    train_idec()
    #                                     for name in net.state_dict():
    #                                         print(name)



use cuda: True
data samples (5110, 8, 2048)
Namespace(batch_size=256, cuda=True, dataset='mnist', gamma=100, in_channels=8, lambda1=0.001, lambda2=0.001, layer1=500, layer2=1600, lr=0.0001, n_clusters=5, n_input=2048, n_z=20, pretrain_path='ae_Adam_multiview.pkl', tol=0.001, update_interval=1)
MAE(
  (ae1): AE(
    (enc_1): Linear(in_features=2048, out_features=500, bias=True)
    (enc_2): Linear(in_features=500, out_features=500, bias=True)
    (enc_3): Linear(in_features=500, out_features=1600, bias=True)
    (z_layer): Linear(in_features=1600, out_features=20, bias=True)
    (dec_1): Linear(in_features=20, out_features=1600, bias=True)
    (dec_2): Linear(in_features=1600, out_features=500, bias=True)
    (dec_3): Linear(in_features=500, out_features=500, bias=True)
    (x_bar_layer): Linear(in_features=500, out_features=2048, bias=True)
  )
  (ae2): AE(
    (enc_1): Linear(in_features=2048, out_features=500, bias=True)
    (enc_2): Linear(in_features=500, out_features=500, bias=Tru

TypeError: can't convert cuda:1 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.